In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%matplotlib inline

In [3]:
from tabulate import tabulate
import sqlite3

In [4]:
con = sqlite3.connect('meetup.db')
cursor = con.cursor()

Let's peek at the schema. Limit to just first 30 rows.

Event can be viewed with URL https://www.meetup.com/{group_name}/events/{event_id}/

For example, https://www.meetup.com/1-5GenAsians/events/281289732/

In [5]:
data = cursor.execute("select * from event limit 30;")
print(tabulate(data.fetchall(), headers=["Event ID", "Status",'Event Date', 'RSVP Count', "Link"], tablefmt='psql'))

+------------+----------+--------------+--------------+-------------------------------------------------------+
|   Event ID | Status   | Event Date   |   RSVP Count | Link                                                  |
|------------+----------+--------------+--------------+-------------------------------------------------------|
|  109370632 | past     | 2013-03-22   |            5 | https://www.meetup.com/1-5GenAsians/events/109370632/ |
|  109868492 | past     | 2013-03-23   |            6 | https://www.meetup.com/1-5GenAsians/events/109868492/ |
|  111080172 | past     | 2013-03-30   |           11 | https://www.meetup.com/1-5GenAsians/events/111080172/ |
|  111088522 | past     | 2013-03-30   |            3 | https://www.meetup.com/1-5GenAsians/events/111088522/ |
|  110749042 | past     | 2013-03-31   |            8 | https://www.meetup.com/1-5GenAsians/events/110749042/ |
|  111871212 | past     | 2013-04-05   |           10 | https://www.meetup.com/1-5GenAsians/events/11187

Let's see some of the most popular events. I define this to be events with the most RSVP.

In [6]:
data = cursor.execute("select event_id, event_date, rsvp_count, link from event order by rsvp_count desc limit 30;")
print(tabulate(data.fetchall(), headers=["Event ID", 'Event Date', 'RSVP Count', 'Link'], tablefmt='psql'))

+------------+--------------+--------------+-------------------------------------------------------+
|   Event ID | Event Date   |   RSVP Count | Link                                                  |
|------------+--------------+--------------+-------------------------------------------------------|
|  219920453 | 2015-03-01   |          109 | https://www.meetup.com/1-5GenAsians/events/219920453/ |
|  212049012 | 2014-11-02   |          105 | https://www.meetup.com/1-5GenAsians/events/212049012/ |
|  233570064 | 2016-10-09   |          102 | https://www.meetup.com/1-5GenAsians/events/233570064/ |
|  230305968 | 2016-04-30   |           85 | https://www.meetup.com/1-5GenAsians/events/230305968/ |
|  266398172 | 2019-12-21   |           83 | https://www.meetup.com/1-5GenAsians/events/266398172/ |
|  248285057 | 2018-03-24   |           82 | https://www.meetup.com/1-5GenAsians/events/248285057/ |
|  250424473 | 2018-06-10   |           80 | https://www.meetup.com/1-5GenAsians/events/250

In [11]:
data = cursor.execute("select count(*) from event;")
print(f"Total number of events is {data.fetchone()[0]}")

Total number of events is 814
